In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ann import ann
import ann_utils
import gd
import maps
import train_utils
import wandb

In [2]:
wandb.login()
project_name = 'cs6910-Assignment-1-MSE'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cs22m068 (cs6910-cs22m068). Use `wandb login --relogin` to force relogin


In [3]:
from keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Normalize data
X_train = X_train / 255.0
X_test = X_test / 255.0

# Meta information about the data.
n_train = len(y_train.flatten())
n_test = len(y_test.flatten())
input_dim = len(X_train[0].flatten())
output_dim = 10

# Split train data to train and validation sets.
ind = [i for i in range(n_train)]
np.random.shuffle(ind)
m = int(n_train * 0.1)
X_valid = X_train[ind[1: m]]
y_valid = y_train[ind[1: m]]
X_train = X_train[ind[m:]]
y_train = y_train[ind[m:]]



2023-03-20 18:48:58.234439: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 18:48:58.357796: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-20 18:48:58.357817: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-20 18:48:59.031202: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [4]:
y_train.shape
y_train[0]

0

In [5]:
def train(config):
    # Hyperparameters:
    epochs = config.epochs
    ann.weight_decay = config.weight_decay
    ann.learning_rate = config.learning_rate

    # ANN architecture
    ann.num_layers = config.num_layers
    ann.hidden_size = config.hidden_size
    ann.batch_size = config.batch_size
    ann.weight_init = maps.weight_init[config.weight_init]
    ann.activation = maps.activation[config.activation]
    ann.d_activation = maps.d_activation[config.activation]

    # Functions
    optimizer = maps.optimizer[config.optimizer]
    ann.loss = 'mean_squared_error'

    nn = ann(input_dim, output_dim)

    optimizer(nn, X_train, y_train, epochs)

    performance = []
    performance.append(train_utils.get_classification_accuracy(nn, X_test, y_test))
    performance.append(train_utils.get_classification_accuracy(nn, X_valid, y_valid))
    performance.append(train_utils.get_loss(nn, X_test, y_test))
    performance.append(train_utils.get_loss(nn, X_valid, y_valid))

    return performance

In [6]:
def main():
    wandb.init(project = project_name)
    performance = train(wandb.config)
    wandb.log({'acc': performance[0], 'val_acc': performance[1], 'loss': performance[2], 'val_loss': performance[3]})

sweep_configuration = {
    'method': 'bayes',
    'metric': {
        'goal': 'minimize',
        'name': 'val_loss'
    },
    'parameters': {
        'epochs': {'values': [5, 10]},
        'num_layers': {'values': [3, 4, 5]},
        'hidden_size': {'values': [32, 64, 128]},
        'weight_decay': {'values': [0, 0.0005, 0.5]},
        'learning_rate': {'values': [0.001, 0.0001]},
        'optimizer': {'values': ['sgd', 'momentum', 'rmsprop', 'adam']},
        'batch_size': {'values': [16, 32, 64]},
        'weight_init': {'values': ['random', 'Xavier']},
        'activation': {'values': ['sigmoid', 'tanh', 'ReLU']},
    }
}

sweep_id = wandb.sweep(sweep = sweep_configuration, project = project_name)
wandb.agent(sweep_id, function = main, count = 90)

Create sweep with ID: 1zeei1ag
Sweep URL: https://wandb.ai/cs6910-cs22m068/CS6910-Assignment-1-MSE/sweeps/1zeei1ag


wandb: Agent Starting Run: nmqiyn9q with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	hidden_size: 32
wandb: 	learning_rate: 0.0001
wandb: 	num_layers: 3
wandb: 	optimizer: adam
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: random
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Run nmqiyn9q errored: ValueError('math domain error')
wandb: ERROR Run nmqiyn9q errored: ValueError('math domain error')
wandb: Agent Starting Run: tuc2oh4b with config:
wandb: 	activation: tanh
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 4
wandb: 	optimizer: momentum
wandb: 	weight_decay: 0.5
wandb: 	weight_init: random
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


acc,▁
loss,▁
val_acc,▁
val_loss,▁
acc,0.1
loss,0.57049
val_acc,0.10185
val_loss,0.57057


wandb: Agent Starting Run: hvkdijvg with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 3
wandb: 	optimizer: momentum
wandb: 	weight_decay: 0.5
wandb: 	weight_init: Xavier
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

/mnt/d/courses/cs6910_dl/CS6910-Assignment-1/src/ann.py:77: RuntimeWarning: overflow encountered in matmul
  al = self.Ws[ann.num_layers] @ h + self.bs[ann.num_layers]
/mnt/d/courses/cs6910_dl/CS6910-Assignment-1/src/ann.py:100: RuntimeWarning: invalid value encountered in matmul
  grad_W = grad_a @ self.h[k-1].T
/mnt/d/courses/cs6910_dl/CS6910-Assignment-1/src/ann.py:102: RuntimeWarning: invalid value encountered in matmul
  grad_h = self.Ws[k].T @ grad_a
/mnt/d/courses/cs6910_dl/CS6910-Assignment-1/src/ann.py:103: RuntimeWarning: invalid value encountered in multiply
  grad_a = grad_h * ann.d_activation(self.a[k-1])


Run sqk7c0lk errored: ValueError('math domain error')
wandb: ERROR Run sqk7c0lk errored: ValueError('math domain error')
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: rnh2lbq7 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 4
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.5
wandb: 	weight_init: random
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Run rnh2lbq7 errored: ValueError('math domain error')
wandb: ERROR Run rnh2lbq7 errored: ValueError('math domain error')
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 6hxhuoz8 with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 5
wandb: 	optimizer: momentum
wandb: 	weight_decay: 0.5
wandb: 	weight_init: random
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


acc,▁
loss,▁
val_acc,▁
val_loss,▁
acc,0.1
loss,-0.33552
val_acc,0.10352
val_loss,-0.33548


wandb: Agent Starting Run: lfw7g2w3 with config:
wandb: 	activation: ReLU
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_size: 32
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 5
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.5
wandb: 	weight_init: random
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
